In [86]:
#imports
import csv
import numpy as np
import string
import pandas as pd

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer

import re

import spacy
nlp_lg = spacy.load('en_core_web_lg')
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dmitry/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [100]:
#binary
data = []
with open('tweet_emotions.csv', mode ='r')as file:
    readed = csv.reader(file)
    count = False
    for line in readed:
        if count:
            data.append(line)
        else:
            count=True
    positive = []
    negative = []
    print(set([x[1] for x in data]))
    print(len([x for x in data if x[1]=="surprise"]))

    count = 0
    for x in data:
        if x[1] == "fun" or x[1] == "enthusiasm" or x[1] == "love" or x[1] == "happiness" or x[1] =="relief":
            positive.append(x)
        if x[1] == "hate" or x[1] == "anger" or x[1] == "empty" or x[1] == "worry" or x[1] == "sadness" or x[1] == "boredom":
            #if count % 6 != 0:
            negative.append(x)
        count +=1
    print(len(negative))
    print(len(positive))
    file_pos = open(r'positive.csv', 'w+', newline ='')
    file_neg = open(r'negative.csv', 'w+', newline ='')
    with file_pos:
        write = csv.writer(file_pos)
        write.writerows(positive)
    with file_neg:
        write=csv.writer(file_neg)
        write.writerows(negative)









{'neutral', 'surprise', 'enthusiasm', 'fun', 'anger', 'empty', 'sadness', 'hate', 'boredom', 'relief', 'worry', 'happiness', 'love'}
2187
16063
13112


In [103]:
df_neg = pd.read_csv('negative.csv')
df_pos = pd.read_csv('positive.csv')





AttributeError: 'DataFrame' object has no attribute 'C2'

Data cleaning functions

In [87]:

def remove_punctuation(line_text):
    line_text_without_punct = re.sub(r'[^\w\s]', '', line_text)
    return line_text_without_punct


def remove_numbers(line_text):
    line_text_without_nums = re.sub(r' \d+', '', line_text)
    return line_text_without_nums

def remove_url(line_text):
    #(https?:\/\/) matches http:// or https://
    #(\s)* optional whitespaces
    #(www\.)? optionally matches www.
    # (\s)* optionally matches whitespaces
    #'((\w|\s)+\.)* matches 0 or more of one or more word characters followed by a period
    #([\w\-\s]+\/)* matches 0 or more of one or more words(or a dash or a space) followed by '\'
    #([\w\-]+) any remaining path at the end of the url followed by an optional ending
    #((\?)?[\w\s]*=\s*[\w\%&]*)* matches ending query params (even with white spaces,etc)

    line_text_without_url = re.sub(r'(https?:\/\/)(\s)*(www\.)?(\s)*((\w|\s)+\.)*([\w\-\s]+\/)*([\w\-]+)((\?)?[\w\s]*=\s*[\w\%&]*)*', '', line_text)
    return line_text_without_url

def remove_username(line_text):
    line_text_without_username = re.sub("([@#][A-Za-z0-9_]+)|(\w+:\/\/\S+)",'', line_text)
    return line_text_without_username

def remove_stopwords(line_text):
    line_text_without_stopwords = ' '.join([word for word in line_text.split()
                                           if word not in (stopwords.words('english'))])
    return line_text_without_stopwords

def translate_abbreviations(line_text):
    text_list = line_text.split()
    with open("slang_abbreviations.txt", 'r') as my_csv_file:
        data_from_file = csv.reader(my_csv_file, delimiter="=")
        for index, _str in enumerate(text_list):
            _str = re.sub('[^a-zA-Z0-9-_.]', '', _str)
            for row in data_from_file:
                if _str.upper() == row[0]:
                    text_list[index] = row[1]
    final_string = ' '.join(text_list)
    return final_string

def remove_hashtags(line_text):
    line_text_without_hashtags = line_text.replace('#','')
    return line_text_without_hashtags

ps = nltk.PorterStemmer()
def stemming(line_text):
    text_list = [ps.stem(word) for word in line_text.split()]
    final_string = ' '.join(text_list)
    return final_string


def clean_line(line_text):
    text_zero = remove_username(line_text)
    text_one = remove_url(text_zero)
    text_two = remove_punctuation(text_one)
    text_three = remove_hashtags(text_two)
    text_four = remove_numbers(text_three)
    text_five = remove_stopwords(text_four)
    text_six = translate_abbreviations(text_five)
    text_final = stemming(text_six)
    return text_final



Cleaning the text and saving cleaned version to new column 'cleaned_content'

In [88]:
df = pd.read_csv("tweet_emotions.csv")
df['cleaned_content'] = df.content.apply(clean_line)


Saving cleaned Dataframe to scv for future

In [89]:
df.to_csv('tweet_emotions_cleaned.csv')

In [90]:
df

,tweet_id,sentiment,content,cleaned_content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...,know listenin bad habit earlier start freakin ...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...,layin n bed headach ughhhhwaitin call
2,1956967696,sadness,Funeral ceremony...gloomy friday...,funer ceremonygloomi friday
3,1956967789,enthusiasm,wants to hang out with friends SOON!,want hang friend soon
4,1956968416,neutral,@dannycastillo We want to trade with someone w...,we want trade someon houston ticket one
...,...,...,...,...
39995,1753918954,neutral,@JohnLloydTaylor,
39996,1753919001,love,Happy Mothers Day All my love,happi mother day all love
39997,1753919005,love,Happy Mother's Day to all the mommies out ther...,happi mother day mommi woman man long your mom...
39998,1753919043,happiness,@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!! PEE...,wassup beauti follow me peep out my new hit si...


function which vectorize text and give the train and test data back

In [91]:
def SpacyVec_data_from_df(df_input):
    df = df_input
    df['text_tokenized'] = df.clened_content.apply(lambda x: nlp_lg(x)) # tokenizing every text in the dataframe

    df.sentiment = pd.Categorical(df.sentiment) # changing type of a column to categorical
    df['Numeric_Labels'] = df.sentiment.cat.codes # saving category codes of column labels to new column

    #doing sentence embedding before splitting, because the order makes no difference
    df['text_vectorized'] = df['text_tokenized'].apply(lambda x: x.vector)

    # splitting data _________________COULD BE CHANGED____________________________________________________
    data_train, data_test, label_train, label_test = train_test_split(df.text_vectorized, df.Numeric_Labels,
                                                                 test_size=0.30,
                                                                 random_state=101, shuffle=True)
   # __________________________________________________________________________________________________________

    # appending all array rows into one and reshaping them from (1,300) shape to (300,1) shape
    data_list_train = [vec.reshape(1,-1) for vec in data_train]
    data_list_test = [vec.reshape(1,-1) for vec in data_test]

    # concentrating array to get them readable by sklearn models
    data_train_reshaped = np.concatenate(data_list_train)
    data_test_reshaped = np.concatenate(data_list_test)

    return data_train_reshaped, data_test_reshaped, label_train, label_test





In [95]:
def TfIdf_data_from_df(df_input):
    df = df_input
    df.sentiment = pd.Categorical(df.sentiment) # changing type of a column to categorical
    df['Numeric_Labels'] = df.sentiment.cat.codes # saving category codes of column labels to new column

    # splitting data _________________COULD BE CHANGED____________________________________________________
    data_train, data_test, label_train, label_test = train_test_split(df.cleaned_content, df.Numeric_Labels, test_size=0.30, random_state=101, shuffle=True)
   # __________________________________________________________________________________________________________

   # vectorizing test and train data with Tfidf
    polarity_tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2))
    polarity_bow_matrix_train = polarity_tfidf_vectorizer.fit_transform(data_train)
    polarity_bow_matrix_test = polarity_tfidf_vectorizer.transform(data_test)


    return polarity_bow_matrix_train, polarity_bow_matrix_test, label_train, label_test


In [96]:
data_train, data_test, label_train, label_test = TfIdf_data_from_df(df)

In [97]:
forest_classifier = RandomForestClassifier()

In [85]:
forest_classifier = RandomForestClassifier()
forest_classifier.fit(data_train, label_train)
test_forest= forest_classifier.predict(data_test)

#evaluationg model and reporting accuracy and multi-class confusion matrix
print('\nEvaluate MLPClassifier:')
print('\nAccuracy: %.3f' % metrics.accuracy_score(label_test, test_forest))
print('\nConfusion Matrix: ')
multilabel_confusion_matrix(label_test, test_forest)


Evaluate MLPClassifier:

Accuracy: 0.330

Confusion Matrix: 


array([[[11966,     0],
        [   34,     0]],

       [[11946,     4],
        [   50,     0]],

       [[11704,    33],
        [  261,     2]],

       [[11775,    13],
        [  212,     0]],

       [[11390,    81],
        [  513,    16]],

       [[ 9408,  1001],
        [ 1109,   482]],

       [[11519,    86],
        [  341,    54]],

       [[10323,   589],
        [  684,   404]],

       [[ 6056,  3322],
        [ 1055,  1567]],

       [[11442,    69],
        [  471,    18]],

       [[ 9867,   517],
        [ 1292,   324]],

       [[11250,   122],
        [  600,    28]],

       [[ 7316,  2201],
        [ 1416,  1067]]])